In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from medmnist import INFO, PathMNIST
import medmnist

# Importación del conjunto de datos

In [ ]:
# Información del dataset
info = INFO['pathmnist']
DataClass = getattr(medmnist, info['python_class'])

In [ ]:
# Transforms (normalización de [0, 255] a [0.0, 1.0])
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

In [ ]:
# Cargar datasets
train_dataset = DataClass(split='train', transform=transform, download=True)
val_dataset = DataClass(split='val', transform=transform, download=True)
test_dataset = DataClass(split='test', transform=transform, download=True)

In [ ]:
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
#Revisamos el tamaño de los conjuntos de datos
print(f"Tamaño del conjunto de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del conjunto de prueba: {len(test_dataset)}")
print(f"Tamaño del conjunto de validación: {len(val_dataset)}")

Tamaño del conjunto de entrenamiento: 89996
Tamaño del conjunto de prueba: 7180
Tamaño del conjunto de validación: 10004


# Modelo maestro

## Prueba 2

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=9):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.fc_layer(x)
        return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN(num_classes=9).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento simple
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.squeeze().long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1, Loss: 0.7479
Epoch 2, Loss: 0.4707
Epoch 3, Loss: 0.3606
Epoch 4, Loss: 0.2967
Epoch 5, Loss: 0.2438
Epoch 6, Loss: 0.2055
Epoch 7, Loss: 0.1717
Epoch 8, Loss: 0.1469
Epoch 9, Loss: 0.1219
Epoch 10, Loss: 0.1054


# Modelo estudiante